In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import backend as keras_backend


def dice_coefficient(true_labels, predicted_labels, epsilon=100):
    true_flat = keras_backend.flatten(true_labels)
    pred_flat = keras_backend.flatten(predicted_labels)
    intersection = keras_backend.sum(true_flat * pred_flat)
    total = keras_backend.sum(true_flat) + keras_backend.sum(pred_flat)
    return (2 * intersection + epsilon) / (total + epsilon)

def dice_loss_function(true_labels, predicted_labels, epsilon=100):
    return -dice_coefficient(true_labels, predicted_labels, epsilon)

def intersection_over_union(true_labels, predicted_labels, epsilon=100):
    intersection = keras_backend.sum(true_labels * predicted_labels)
    union = keras_backend.sum(true_labels + predicted_labels)
    return (intersection + epsilon) / (union - intersection + epsilon)


model = tf.keras.models.load_model("/content/drive/MyDrive/best_model.keras", custom_objects={
    'dice_loss_function': dice_loss_function,
    'intersection_over_union': intersection_over_union,
    'dice_coefficient': dice_coefficient
})

def predict(image):
    image = image.convert("L").resize((128, 128))
    img_array = np.array(image) / 255.0
    img_array = img_array[np.newaxis, :, :, np.newaxis]
    pred = model.predict(img_array)[0, :, :, 0]
    mask = (pred > 0.7).astype(np.uint8)

    diagnosis = "Tumor Detected" if np.sum(mask) > 100 else "No Tumor Detected"
    return diagnosis, Image.fromarray(mask * 255)




In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Brain MRI"),
    outputs=[gr.Text(label="Diagnosis Result"), gr.Image(type="pil", label="Predicted Mask")],
    title="Brain Tumor Detection App",
    allow_flagging="never"
)

interface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://194ae4f76f40b9b723.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
